This Notebook will be used for the Coursera IBM Datasscience Capstone project

In [93]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#  Question1 : Scrape the data from the Wikipedia URL and create the pandas data frame.

In [29]:
 # Scraping the data.
import requests
import lxml.html as lh
import pandas as pd
 from urllib.request import urlopen
 url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
 #Create a handle, page, to handle the contents of the website
 page = requests.get(url)
 #Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]
print ('len(T)',[len(T) for T in tr_elements[:12]])
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"' %(i,name))
    col.append((name,[]))
    
    
    #Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        print ('len(T)!=3')
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        print ('in for')
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        
        [len(C) for (title,C) in col]
        print ('col', col)
        

    

len(T) [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"
in for
col [('Postal Code\n', ['M1A\n']), ('Borough\n', []), ('Neighbourhood\n', [])]
in for
col [('Postal Code\n', ['M1A\n']), ('Borough\n', ['Not assigned\n']), ('Neighbourhood\n', [])]
in for
col [('Postal Code\n', ['M1A\n']), ('Borough\n', ['Not assigned\n']), ('Neighbourhood\n', ['Not assigned\n'])]
in for
col [('Postal Code\n', ['M1A\n', 'M2A\n']), ('Borough\n', ['Not assigned\n']), ('Neighbourhood\n', ['Not assigned\n'])]
in for
col [('Postal Code\n', ['M1A\n', 'M2A\n']), ('Borough\n', ['Not assigned\n', 'Not assigned\n']), ('Neighbourhood\n', ['Not assigned\n'])]
in for
col [('Postal Code\n', ['M1A\n', 'M2A\n']), ('Borough\n', ['Not assigned\n', 'Not assigned\n']), ('Neighbourhood\n', ['Not assigned\n', 'Not assigned\n'])]
in for
col [('Postal Code\n', ['M1A\n', 'M2A\n', 'M3A\n']), ('Borough\n', ['Not assigned\n', 'Not assigned\n']), ('Neighbourhood\n', ['Not assigned\n', 'Not assigned\

In [48]:
# Create the Dataframe from the col array
Dict={title:column for (title,column) in col}
#print ('Dict', Dict)
df=pd.DataFrame(Dict)
df = df.replace({'\n':''}, regex=True)
df.columns=df.columns.str.replace('\n','')
df = df[df['Borough'] != 'Not assigned']
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', 1, df['Neighbourhood'])
df.head()


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [28]:
# Create a Canada postal csv file from the dataframe and store in IBM watson object storge.
#df.to_csv('canpostalcd')
from project_lib import Project
project =Project(project_id='6c00565f-d54b-4419-8ea0-5422c90cb00a', project_access_token='p-4c2fc97e8b68c62358d8689ba87a5c0548556576')
project.save_data(file_name = "canpostalcd.csv",data = df.to_csv(index=False))
print ('canpostalcd csv created')

canpostalcd csv created


In [19]:
#!pip install geocoder

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('North York, CA')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print ('lat {}, lon {}'.format(latitude,longitude))

In [6]:
 import geocoder
 g = geocoder.google('Mountain View, CA')
 g.latlng


KeyboardInterrupt: 

##  Question2: Data frame with latitude and longitudes of Canada postal codes.
This was done using the csv file bcoause the geocode was not responsindg and returning data even after several attempts.

In [11]:
# Read the canada postal geo codes from csv file
df_can = pd.read_csv('https://cocl.us/Geospatial_data',   )
print ('Can geo codes Data read into a pandas dataframe!')

Can geo codes Data read into a pandas dataframe!


In [12]:
df_can.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [47]:
# Merge the origial dataframe created using data scrapped from the wikipedia site.
# The how =left parameter ensures that postal code data is maintained in the new dataframe even if the lat, lon match is not found for the postal code in the geo csv file.
dfinal = df.merge(df_can, on="Postal Code", how="left")
dfinal.head()

,Postal Code,Borough,Neighbourhood,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [26]:
# Create a Canada postal and geocode csv file from the dataframe and store in IBM watson object storge.
from project_lib import Project
project =Project(project_id='6c00565f-d54b-4419-8ea0-5422c90cb00a', project_access_token='p-4c2fc97e8b68c62358d8689ba87a5c0548556576')
project.save_data(file_name = "canpostalgeocd.csv",data = df.to_csv(index=False))
print ('csv can geocode created')  

csv can geocode created


## Question3: Explore and cluster the neighborhoods in Downtown Toronto.

In [41]:
# check the original dataframe
neighborhoods = dfinal
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 104 neighborhoods.


In [49]:
# Slice the original dataframe and create a new dataframe of the Downtown Toronto data.
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Downtown Toronto')].reset_index(drop=True)
toronto_data.columns=toronto_data.columns.str.replace('Neighbourhood','Neighborhood')
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [105]:
# testing for one latitude and longitude.
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [106]:
# Create the foursqare API request URL. The client_id and client_secret were created by opening a foursquare account.
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=X5RIPZ3YIBUWAC2RXUW4Z1Z2QMMUCAX5BS14VOZYWMWXZZCW&client_secret=Z0ALVD55KE4MOP5R2OMSO0IWY3CQNLTLTP1EGDIPMWNEINHT&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [107]:
# Get the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ff20637814c6a0b9682dd50'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

In [117]:
# Create the foursqare API request URL. The client_id and client_secret were created by opening a foursquare account.
CLIENT_ID = 'X5RIPZ3YIBUWAC2RXUW4Z1Z2QMMUCAX5BS14VOZYWMWXZZCW' # your Foursquare ID
CLIENT_SECRET = 'Z0ALVD55KE4MOP5R2OMSO0IWY3CQNLTLTP1EGDIPMWNEINHT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

# Function to get the categories for all latitude and longitudes.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('name:', name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    print('venues_list:', venues_list)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Your credentails:
CLIENT_ID: X5RIPZ3YIBUWAC2RXUW4Z1Z2QMMUCAX5BS14VOZYWMWXZZCW
CLIENT_SECRET:Z0ALVD55KE4MOP5R2OMSO0IWY3CQNLTLTP1EGDIPMWNEINHT


In [109]:
# Create the foursqare API request URL. The client_id and client_secret were created by opening a foursquare account.
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=X5RIPZ3YIBUWAC2RXUW4Z1Z2QMMUCAX5BS14VOZYWMWXZZCW&client_secret=Z0ALVD55KE4MOP5R2OMSO0IWY3CQNLTLTP1EGDIPMWNEINHT&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [118]:
dn_toronto_data_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

name: Regent Park, Harbourfront
name: Queen's Park, Ontario Provincial Government
name: Garden District, Ryerson
name: St. James Town
name: Berczy Park
name: Central Bay Street
name: Christie
name: Richmond, Adelaide, King
name: Harbourfront East, Union Station, Toronto Islands
name: Toronto Dominion Centre, Design Exchange
name: Commerce Court, Victoria Hotel
name: University of Toronto, Harbord
name: Kensington Market, Chinatown, Grange Park
name: CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
name: Rosedale
name: Stn A PO Boxes
name: St. James Town, Cabbagetown
name: First Canadian Place, Underground city
name: Church and Wellesley
venues_list: [[('Regent Park, Harbourfront', 43.6542599, -79.3606359, 'Roselle Desserts', 43.653446723052674, -79.3620167174383, 'Bakery'), ('Regent Park, Harbourfront', 43.6542599, -79.3606359, 'Tandem Coffee', 43.65355870959944, -79.36180945913513, 'Coffee Shop'), ('Regent Park, Harbourfront', 

In [110]:
print(dn_toronto_data_venues.shape)
dn_toronto_data_venues.head()

(1233, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [116]:
# Save the final dataframe as csv so that it can be used in the capstone final project.
project.save_data(file_name = "candntorontovenues.csv",data = df.to_csv(index=False))
print ('csv candntorontovenues created')

csv candntorontovenues created


In [55]:
dn_toronto_data_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [111]:
print('There are {} uniques categories.'.format(len(dn_toronto_data_venues['Venue Category'].unique())))

There are 209 uniques categories.


In [104]:
# one hot encoding- This is to convet the Categories in to columns havinf numeric values.
dn_toronto_onehot = pd.get_dummies(dn_toronto_data_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dn_toronto_onehot['Neighborhood'] = dn_toronto_data_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dn_toronto_onehot.columns[-1]] + list(dn_toronto_onehot.columns[:-1])
dn_toronto_onehot = dn_toronto_onehot[fixed_columns]

dn_toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

In [112]:
dn_toronto_onehot.shape

(1233, 209)

In [67]:
dn_toronto_grouped = dn_toronto_onehot.groupby('Neighborhood').mean().reset_index()
dn_toronto_grouped.shape

(19, 209)

In [68]:
num_top_venues = 5

for hood in dn_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dn_toronto_grouped[dn_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2          Restaurant  0.03
3  Seafood Restaurant  0.03
4              Bakery  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2  Airport Terminal  0.12
3     Boat or Ferry  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.18
1      Sandwich Place  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4        Burger Joint  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3      Nightclub  0.06
4     Restaurant  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.06
3           Restaurant  0.04
4        

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [72]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dn_toronto_grouped['Neighborhood']

for ind in np.arange(dn_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dn_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Eastern European Restaurant,Gourmet Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Plane,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Bar
2,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Miscellaneous Shop,Japanese Restaurant
3,Christie,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Hotel,Café,Yoga Studio,Mediterranean Restaurant,Men's Store


# Clustering Kmeans

In [76]:
# set number of clusters
kclusters = 5 # This will create 5 clusters

dn_toronto_grouped_clustering = dn_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dn_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 4], dtype=int32)

In [81]:
# add clustering labels
# labels are already inserted in the above code.
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
dn_toronto_merged = toronto_data

# merge dn_toronto_merged with toronto_data to add latitude/longitude for each neighborhood
dn_toronto_merged = dn_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dn_toronto_merged.head() # check the last columns!
#dn_toronto_merged['Cluster Labels'].unique

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Café,Bakery,Pub,Breakfast Spot,Theater,Restaurant,Yoga Studio,Farmers Market
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Bar,Italian Restaurant,Japanese Restaurant,Music Venue,Sandwich Place,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Department Store,Creperie,Cosmetics Shop,Gym,Clothing Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Eastern European Restaurant,Gourmet Shop


In [98]:
# Get the Latitude and longitude of Downtown Toronto, Country code CA
address = 'Downtown Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
dn_toronto_latitude = location.latitude
dn_toronto_longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6563221, -79.3809161.


In [99]:
# create map
map_clusters = folium.Map(location=[dn_toronto_latitude, dn_toronto_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dn_toronto_merged['Latitude'], dn_toronto_merged['Longitude'], dn_toronto_merged['Neighborhood'], dn_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Print the Clusters

In [115]:
# get the 2nd column (index=1) and columns from 5 to the total number of columns (dn_toronto_merged.shape[1])
# do this for all 5 clusters.
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 0, dn_toronto_merged.columns[[1] + list(range(5, dn_toronto_merged.shape[1]))]]

dn_toronto_merged.shape[1]: (19, 16)


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,0,Airport Service,Airport Lounge,Airport Terminal,Boutique,Plane,Airport,Airport Food Court,Airport Gate,Boat or Ferry,Bar


In [100]:
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 1, dn_toronto_merged.columns[[1] + list(range(5, dn_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Café,Bakery,Pub,Breakfast Spot,Theater,Restaurant,Yoga Studio,Farmers Market
1,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Bar,Italian Restaurant,Japanese Restaurant,Music Venue,Sandwich Place,Distribution Center
2,Downtown Toronto,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Italian Restaurant,Bookstore
3,Downtown Toronto,1,Coffee Shop,Café,American Restaurant,Gastropub,Cocktail Bar,Department Store,Creperie,Cosmetics Shop,Gym,Clothing Store
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Bakery,Beer Bar,Restaurant,Eastern European Restaurant,Gourmet Shop
5,Downtown Toronto,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Miscellaneous Shop,Japanese Restaurant
7,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Deli / Bodega,Thai Restaurant,Gym,Pizza Place,Clothing Store,Bookstore
8,Downtown Toronto,1,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Baseball Stadium
9,Downtown Toronto,1,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Italian Restaurant,Salad Place,Seafood Restaurant,American Restaurant,Asian Restaurant
10,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Italian Restaurant


In [101]:
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 2, dn_toronto_merged.columns[[1] + list(range(5, dn_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,2,Grocery Store,Café,Park,Athletics & Sports,Italian Restaurant,Nightclub,Candy Store,Restaurant,Baby Store,Coffee Shop


In [102]:
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 3, dn_toronto_merged.columns[[1] + list(range(5, dn_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,3,Park,Trail,Playground,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [103]:
dn_toronto_merged.loc[dn_toronto_merged['Cluster Labels'] == 4, dn_toronto_merged.columns[[1] + list(range(5, dn_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Downtown Toronto,4,Café,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Yoga Studio,Beer Bar,College Gym,Sandwich Place
12,Downtown Toronto,4,Café,Mexican Restaurant,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Coffee Shop,Farmers Market,Dessert Shop,Comfort Food Restaurant,Caribbean Restaurant,Gaming Cafe
